In [ ]:
using Plots; plotlyjs()

fntsm = Plots.font("sans-serif", 10.0)
fntlg = Plots.font("sans-serif", 14.0)
Plots.default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm)
Plots.default(size=(720,540))

## OCPI2 status update: Imagine upgrades
<br>
<br>
#### <center>Cody
#### <center>Lab Meeting
#### <center>13 June 2017

 <center>![noimg](microscope2.png)</center>

 <center>![noimg](optophys3.png)</center>

### Ways to control hardware
 1. With software only.  **Problem:** Inconsistent timing
 2. Analog signals (Voltage)
 <center>![noimg](generic_sawtooth.png)</center>
 3. Digital signals (Voltage)
 <center>![noimg](generic_square_wave.jpg)</center>

### Experiments enabled by hardware timing
1. High-speed volumetric timeseries (my project)
2. Sub-stack stimulus delivery timing
3. Align images with concurrent recordings (ephys, mic, etc)
4. Long experiments with intermittent recording
5. Concurrent point-and-shoot optogenetics

### OCPI2 output channels
- 4 analog outputs, 0-10V range
    - 2 reserved for positioner control
    - 2 will be used for galvo mirrors
    - Max analog output rate (4 channels): 1.25 MS/s
- 23 digital outputs, 3.3V TTL
    - 6 reserved for laser control
    - 2 reserved for camera control
    - 15 available to users, **may reduce to 11**
    - Max digital output rate: 10 MS/s
- Requirement: **all** analog and digital IO rates are equal



<center>![noimg](imagine_waveform_fullgui.jpg)</center>

<center>![noimg](imagine_waveform.jpg)</center>

## Load the package

In [ ]:
using ImagineInterface

import Unitful: μm, s

## Show currently supported rigs

In [ ]:
ImagineInterface.RIGS

## Decide on a sampling rate
### (this applies to both input and output signals)

In [ ]:
sample_rate = 50000s^-1 #analog output samples per second

### Choose a rig and create an empty set of commands

In [ ]:
rig = "ocpi-2"
ocpi2 = rigtemplate(rig; sample_rate = sample_rate);
@show typeof(ocpi2);
@show length(ocpi2);
@show ocpi2[1];
@show ocpi2[3];

## Extracting subsets of commands

In [ ]:
getpositioners(ocpi2)

In [ ]:
getanalog(ocpi2)

In [ ]:
getcameras(ocpi2)

In [ ]:
getlasers(ocpi2)

In [ ]:
getstimuli(ocpi2)

## Creating a vector of positioner samples

In [ ]:
sweep_up = [0.0:0.1:800.0...] * Unitful.μm

## Visualize it

In [ ]:
using Plots, UnitfulPlots
plotlyjs()
plot(sweep_up)

## Add these samples to a positioner command

In [ ]:
pos = getpositioners(ocpi2)[1] #get the axial positioner
append!(pos, "sweep_up", sweep_up)

## If you want to retrieve the samples from the command again...

In [ ]:
sweep_up2 = decompress(pos, "sweep_up")
plot(sweep_up2)

## Also decompress using raw sample indices or a time interval

In [ ]:
#sample indices
some_samps = decompress(pos, 50, 1000)
#time
more_samps = decompress(pos, 0.01s, 0.12s)
plot(more_samps)

## Other ways to add and remove samples
### Remove the last appended sequence

In [ ]:
pop!(pos);
@show pos

### Append it again

In [ ]:
append!(pos, "sweep_up")
@show pos
pop!(pos); #back to empty

## An easier way to generate commands

### First decide on some parameters...

In [ ]:
pmin = 0.0*μm #Piezo start position
pmax = 200.0*μm #Piezo stop position
stack_img_time = 1.0s #Time to complete the imaging sweep with the piezo
reset_time = 0.5s #Time to reset piezo to starting position
z_spacing = 3.1μm #The space between slices in the z-stack
z_pad = 5.0μm #Set this greater than 0 if you only want to only take slices in a central region of the sweep
exp_time = 0.011s  #Exposure time of the camera
flash_frac = 0.1; #Fraction of time to keep laser on during exposure

## By the way, if you want to know how fast the camera can go...

In [ ]:
hpix = 1000 #horizontal ROI size
vpix = 1000 #vertical ROI size
@show mx_f = max_framerate(rig, hpix,vpix) #frames per second
@show mn_exp = 1/mx_f; #Minimum possible exposure time

## And if you forget the maximum ROI size of the camera...

In [ ]:
hmax, vmax = chip_size(rig)

## Now generate samples for a stack

In [ ]:
stack_samps = gen_unidirectional_stack(pmin, pmax, z_spacing, stack_img_time, reset_time, exp_time, sample_rate, flash_frac; z_pad = z_pad)

### The "positioner", "camera", and "laser" entries hold vectors of samples
### Now let's append them to their respective commands

In [ ]:
las1 = getlasers(ocpi2)[1]
cam1 = getcameras(ocpi2)[1]
append!(pos, "uni_stack_pos", stack_samps["positioner"])
append!(las1, "uni_stack_las1", stack_samps["laser"])
append!(cam1, "uni_stack_cam1", stack_samps["camera"]);
nframes = stack_samps["nframes"]; #store this for later

## Visualizing the commands

In [ ]:
plot([pos;las1;cam1])

## When you are satisfied with the single stack waveform...

In [ ]:
replicate!(pos, 4)
replicate!(las1, 4)
replicate!(cam1, 4)
plot([pos; las1; cam1])

## When ready, write the commands to file

In [ ]:
nframes = stack_samps["nframes"];
nstacks = 5
write_commands("test_uni.json", ocpi2, nstacks, nframes, exp_time; isbidi = false)

## A bidirectional recording

In [ ]:
bidi_samps = gen_bidirectional_stack(pmin, pmax, z_spacing, stack_img_time, exp_time, sample_rate, flash_frac; z_pad = z_pad)

### Empty our previous commands and add the bidirectional ones

In [ ]:
empty!(pos)
empty!(las1)
empty!(cam1)
append!(pos, "bidi_stack_pos", bidi_samps["positioner"])
append!(las1, "bidi_stack_las1", bidi_samps["laser"])
append!(cam1, "bidi_stack_cam1", bidi_samps["camera"]);
nframes = bidi_samps["nframes"]
plot(ocpi2)

In [ ]:
replicate!(pos, 4)
replicate!(las1, 4)
replicate!(cam1, 4)
plot(ocpi2)

### Another example: alternate cameras during bidi imaging

In [ ]:
bidi_samps_alt = gen_bidirectional_stack(pmin,
                                    pmax,
                                    z_spacing,
                                    stack_img_time,
                                    exp_time,
                                    sample_rate,
                                    flash_frac;
                                    z_pad = z_pad,
                                    alternate_cameras = true)

In [ ]:
empty!(pos; clear_library = true)
empty!(las1; clear_library = true)
empty!(cam1; clear_library = true)
las2 = getlasers(ocpi2)[2]
cam2 = getcameras(ocpi2)[2]
append!(pos, "bidi_stack_pos", bidi_samps_alt["positioner"])
append!(las1, "bidi_stack_las1", bidi_samps_alt["laser_fwd"])
append!(las2, "bidi_stack_las2", bidi_samps_alt["laser_back"])
append!(cam1, "bidi_stack_cam1", bidi_samps_alt["camera_fwd"]);
append!(cam2, "bidi_stack_cam2", bidi_samps_alt["camera_back"]);
plot(ocpi2)

## What about inputs?
#### We're not quite finished with this yet
#### We plan to support the following:
1. Recording exposure status (digital) for each camera
2. Recording piezo position (analog)
3. Additional user-specified analog and digital channels
4. `input_template("ocpi-2"; samprate = 50000s^-1)`

### OCPI2 extra input channels
- Analog: AI2 ~ AI31
- Digital: P0.26 ~ P0.31

_Remember that max total sample rate for OCPI2 is 2.8 MS/s_


# The end